In [1]:
# 获得 id
# http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=%s' % term_name

# http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=cancer&retstart=3182080&retmax=100
# http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=212403,4584127

from common import *
from lxml.etree import *
import lxml.etree
import math
import re
import pymysql



def esearch(term, retstart, retmax, db='pubmed'):
    '''
    根据 term 查找数据库, 返回 xml
    :param term: 搜索的条目
    :param retstart: 起始位置
    :param retmax:  长度
    :param db: 数据库, 默认为pubmed
    :return: xml
    '''
    term = urllib.parse.quote(term.encode('utf-8', 'replace'))
    # urllib.parse.quote(url.encode('utf-8', 'replace'))
    str_url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=%s&term=%s&retstart=%d&retmax=%d' \
              % (db, term, retstart, retmax)
    request = urllib.request.Request(str_url)
    xml = urllib.request.urlopen(request).read()
    # xml = open_url(str_url, encode='utf-8')
    root = lxml.etree.XML(xml)
    return root


def get_id(term, retstart, retmax, db='pubmed'):
    '''
    返回处理后得到的id
    :param term:  搜索term
    :param retstart:
    :param retmax:
    :param db:
    :return: 返回id列表
    '''
    root = esearch(term, retstart, retmax, db)
    id_list = root.xpath('IdList/Id')
    ids = [id.text for id in id_list]
    return ids


def get_id_count(term, db='pubmed'):
    '''
    得到该 term 下的文章数目
    :param term:
    :param db:
    :return:
    '''
    root = esearch(term, 1, 1, db)
    # print(root.tostring())
    count = root.xpath('Count')[0].text
    return count


def query_id(term, retmax=100, db='pubmed'):
    '''
    核心函数
    :param term:
    :param retmax: 每页显示的id数目
    :param db:
    :return:
    '''
    count = int(get_id_count(term, db=db))
    if count > int(retmax):
        id_list = []
        for ii in range(math.ceil(count / retmax)):
            try:
                ids = get_id(term, ii * retmax, retmax, db=db)
                print('/'.join([str(ii), str(math.ceil(count / retmax))]) )
                for id in ids:
                    cu_mysql.execute('insert into pubmed_id_all values(%s)' , id)
                con_mysql.commit()
            except Exception as e:
                print(e)
                cu_mysql.execute('insert into error_pubmed_id_all values(%s)', (ii) )
                con_mysql.commit()
            # id_list += ids
        # return id_list
    else:
        id_list = get_id(term, 0, count, db=db)
        for id in id_list:
            cu_mysql.execute('insert into pubmed_id_all values(%s)', id)
            con_mysql.commit()




def efetch(ids, db='pubmed'):
    '''
    根据id列表获取摘要信息
    :param ids:
    :param db:
    :return:
    '''
    str_url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=%s&id=%s' % (db, ids)
    if db=='pubmed':
        str_url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=%s&id=%s&retmode=text&rettype=xml' % (db, ids)

    request = urllib.request.Request(str_url)
    xml = urllib.request.urlopen(request).read()
    root = lxml.etree.XML(xml)
    return root


def  get_xpath_0(obj, xpath):
    tmp = obj.xpath(xpath)
    if tmp:
        return tmp[0].text
    else:
        return '0'

# bug 太多弃用
def process_pmc(term):
    '''
    处理pmc数据库文章
    :param term:
    :return:
    '''
    try:
        root = efetch(term, db='pmc')
        articles = root.xpath('//article')
        # 对每一篇文章进行处理
        for article in articles:
            journal = article.xpath('.//journal-title')[0].text
    #         journal-id journal-id-type="nlm-journal-id"
            if article.xpath('.//journal-id[@journal-id-type="nlm-journal-id"]'):
                journal_nlm_id = article.xpath('.//journal-id[@journal-id-type="nlm-journal-id"]')[0].text
            else:
                journal_nlm_id = '0'
            issn = article.xpath('.//issn')[0].text
    #         print(issn)
            title = article.xpath('.//article-title')[0].text
            subject = article.xpath('.//subject')[0].text
            pmid_tmp = article.xpath('.//article-id[@pub-id-type="pmid"]')
            if pmid_tmp:
                pmid = pmid_tmp[0].text
#                 pmid_tmp = get_xpath_0(article, './/article-id[@pub-id-type="pmid"]')
            else:
                pmid = '0'
            pmc_id = article.xpath('.//article-id[@pub-id-type="pmc"]')[0].text
            doi = article.xpath('.//article-id[@pub-id-type="doi"]')[0].text
            authors = article.xpath('.//contrib[@contrib-type="author"]')
            year = article.xpath('.//pub-date//year')[0].text
            # 文章信息

            keywords = article.xpath('.//kwd-group//kwd')
            keyword_list = [kw.text for kw in keywords if kw.text]
            keyword_join = ','.join(keyword_list)
    #         ariticl_info = [pmc_id, pmid, doi, journal,journal_nlm_id, issn, title, subject, year, keyword_join]
    #         print(ariticl_info)
            cu_mysql.execute('insert into pmc_article(pmc_id, pmid, doi, journal,journal_nlm_id, issn, title, subject, year, keyword_join) \
            values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)',
                     (pmc_id, pmid, doi, journal,journal_nlm_id, issn, title, subject, year, keyword_join))
            aff_list = article.xpath('.//aff[@id]')
            aff_dict = {}
            # 获取作者单位列表
            for aff in aff_list:
                aff_str = tostring(aff)
                # 此处正则表达式， 因为pmc有</label> 和 <label/>两种，为兼顾两者，暂时采用 > 代替
                aff_str_tmp = re.compile(r'(?<=>)[\s\S]*(?=</aff>)').findall(aff_str.decode('utf-8'))
                if aff_str_tmp:
                    aff_str = aff_str_tmp[0]
                aff_id = aff.get('id')
                aff_dict[aff_id] = aff_str

                    # 通讯作者， 邮箱， PMC有两类，一类 是跟其他作者一起， 但用<xref ref-type="corresp" rid="CR1">*</xref> 区别
            #<contrib contrib-type="author">
    #         <name>
    #         <surname>Jiang</surname>
    #         <given-names>Liwen</given-names>
    #         </name>
    #         <xref ref-type="aff" rid="A1">a</xref>
    #         <xref ref-type="aff" rid="A3">c</xref>
    #         <xref ref-type="corresp" rid="CR1">*</xref>
    #         </contrib>
    # 另一类
    #     <contrib id="A3" corresp="yes" contrib-type="author">

    # 第一类
    # 获取通讯作者邮箱
            corresp_list = article.xpath('.//corresp[@id]')
            corresp_dict = {}
            for corresp_info in corresp_list:
                corresp_id = corresp_info.get('id')
                corresp_dict[corresp_id] = corresp_info.xpath('.//email')[0].text


            author_index = 0
            for author in authors:
                author_index += 1
                surname = author.xpath('.//name//surname')[0].text
                given_names = author.xpath('.//name//given-names')[0].text
    #             full_name = surname + given_names
                email_tmp = author.xpath('.//email')
                if email_tmp:
                    email = email_tmp[0].text
                else:
                    email='NA'
                # 是否通讯作者
                # <xref ref-type="corresp" rid="CR1">*</xref>
    #             corres = author.xpath('.//x')
                corresp = 0
                xref_corresp = author.xpath(".//xref[@ref-type='corresp']") # [@lang='eng']
                if  xref_corresp:            
                    corresp = 1
                    corresp_id = xref_corresp[0].get('rid')
                    email = corresp_dict[corresp_id]
                # 通讯作者邮箱
                # 获取单位
                xref = author.xpath(".//xref[@ref-type='aff']")
                for ii in range(len(xref)):
                    if(xref[ii].get('ref-type') == 'aff' ):
                        aff_name = aff_dict[xref[ii].get('rid')]
    #                     print(aff_name)
                    cu_mysql.execute('insert into pmc_authors(pmc_id, author_index, surname, given_names, email, corresp, aff_name) \
                                     values(%s, %s, %s, %s, %s, %s, %s)',
                                     (pmc_id, author_index, surname, given_names, email, corresp, aff_name))

    #       print(pmc_id, author_index, surname, given_names, email, corresp, aff_name)


    #      以下为第二类
            corresp_authors = article.xpath('.//contrib[@corresp="yes"]')
            for corresp_author in corresp_authors:
                surname = corresp_author.xpath('.//surname')[0].text
                given_names = corresp_author.xpath('.//given-names')[0].text
    #             corresp_name = surname + given_names
                email_tmp = corresp_author.xpath('.//email')
                if email_tmp:
                    email = email_tmp[0].text
                else:
                    email='NA'
                # pmc_id, surname, given_names, email
                cu_mysql.execute('insert into pmc_corresp_author(pmc_id, surname, given_names, email) \
                     values(%s, %s, %s, %s)',
                     (pmc_id, surname, given_names, email))
        con_mysql.commit()
    except Exception as e:
        print(e)
        con_mysql.rollback()
        cu_mysql.execute('insert into error_pmc values(%s, %s)', (term, str(e)))
        con_mysql.commit()


def process_pmc2(term):
    '''
    处理pmc数据库文章
    :param term:
    :return:
    '''
    try:
        root = efetch(term, db='pmc')
        articles = root.xpath('//article')
        # 对每一篇文章进行处理
        for article in articles:
            journal = get_xpath_0(article, './/journal-title')
    #         journal-id journal-id-type="nlm-journal-id"
            if article.xpath('.//journal-id[@journal-id-type="nlm-journal-id"]'):
                journal_nlm_id = article.xpath('.//journal-id[@journal-id-type="nlm-journal-id"]')[0].text
            else:
                journal_nlm_id = '0'
            issn = get_xpath_0(article, './/issn')
    #         print(issn)
            title = tostring(article.xpath('.//article-title')[0])
            strinfo = re.compile('<[^>]*>')
            title = strinfo.sub('',title.decode('utf-8'))
        #get_xpath_0(article, './/article-title')
            subject = get_xpath_0(article, './/subject')
            pmid = get_xpath_0(article, './/article-id[@pub-id-type="pmid"]')

            pmc_id = get_xpath_0(article, './/article-id[@pub-id-type="pmc"]')
            doi = get_xpath_0(article, './/article-id[@pub-id-type="doi"]')
            authors = article.xpath('.//contrib[@contrib-type="author"]')
            year = get_xpath_0(article,'.//pub-date//year')
            # 文章信息

            keywords = article.xpath('.//kwd-group//kwd')
            keyword_list = [kw.text for kw in keywords if kw.text]
            keyword_join= '|'.join(keyword_list)
    #         ariticl_info = [pmc_id, pmid, doi, journal,journal_nlm_id, issn, title, subject, year, keyword_join]
    #         print(ariticl_info)

    #             print(pmc_id, pmid, doi, journal,journal_nlm_id, issn, title, subject, year, keyword_join)
            aff_list = article.xpath('.//aff[@id]')
            aff_dict = {}
            # 获取作者单位列表
            first_aff = '0'
            first_aff_flag = True
            for aff in aff_list:
                aff_str = tostring(aff)
                # 此处正则表达式， 因为pmc有</label> 和 <label/>两种，为兼顾两者，暂时采用 > 代替
                aff_str_tmp = re.compile(r'(?<=>)[\s\S]*(?=</aff>)').findall(aff_str.decode('utf-8'))

                if aff_str_tmp:
                    aff_str = aff_str_tmp[0]
    #                     strinfo = re.compile('<[^>]*>')
    #                     aff_str = strinfo.sub('',aff_str)
                else:
                    aff_str = '0'
                aff_id = aff.get('id')
                aff_dict[aff_id] = aff_str
                if(first_aff_flag):
                    first_aff = aff_str
                    first_aff_flag = False
            cu_mysql.execute('insert into pmc_article2(pmc_id, pmid, doi, journal,first_aff, issn, title, subject, year, keyword_join) \
                values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)',
                     (pmc_id, pmid, doi, journal,first_aff, issn, title, subject, year, keyword_join))
                    # 通讯作者， 邮箱， PMC有两类，一类 是跟其他作者一起， 但用<xref ref-type="corresp" rid="CR1">*</xref> 区别
            #<contrib contrib-type="author">
    #         <name>
    #         <surname>Jiang</surname>
    #         <given-names>Liwen</given-names>
    #         </name>
    #         <xref ref-type="aff" rid="A1">a</xref>
    #         <xref ref-type="aff" rid="A3">c</xref>
    #         <xref ref-type="corresp" rid="CR1">*</xref>
    #         </contrib>
    # 另一类
    #     <contrib id="A3" corresp="yes" contrib-type="author">

    # 第一类
    # 获取通讯作者邮箱

            corresp_dict = {}
            if article.xpath('.//author-notes'):
                corresp_list = article.xpath('.//author-notes')[0]
                for corresp_info in list(corresp_list):
                    corresp_id = corresp_info.get('id')
                    corresp_dict[corresp_id] =get_xpath_0(corresp_info, './/email')


            author_index = 0
            for author in authors:
                author_index += 1
                surname = get_xpath_0(author, './/name//surname')
                given_names = get_xpath_0(author,'.//name//given-names')
    #             full_name = surname + given_names
                email_tmp = author.xpath('.//email')
                if email_tmp:
                    email = email_tmp[0].text
                else:
                    email='NA'
                # 是否通讯作者
                # <xref ref-type="corresp" rid="CR1">*</xref>
    #             corres = author.xpath('.//x')
                corresp = 0
                xref_corresp = author.xpath(".//xref[@ref-type='corresp']") # [@lang='eng']
                if  xref_corresp:            
                    corresp = 1
                    corresp_id = xref_corresp[0].get('rid')
                    email = corresp_dict[corresp_id]
                # 通讯作者邮箱
                # 获取单位
                xref = author.xpath(".//xref[@ref-type='aff']")
                for ii in range(len(xref)):
                    if(xref[ii].get('ref-type') == 'aff' ):
                        aff_flag_list = xref[ii].get('rid').split(' ') # 存在 'a1 a2' 这种情况,所以需要分割后处理
                        for aff_flag in aff_flag_list:
                            aff_name = aff_dict[aff_flag]
                            cu_mysql.execute('insert into pmc_authors2(pmc_id, author_index, surname, given_names, email, corresp, aff_name) \
                                             values(%s, %s, %s, %s, %s, %s, %s)',
                                             (pmc_id, author_index, surname, given_names, email, corresp, aff_name))



    #      以下为第二类
            corresp_authors = article.xpath('.//contrib[@corresp="yes"]')
            for corresp_author in corresp_authors:
                surname = get_xpath_0(corresp_author, './/surname')
                given_names = get_xpath_0(corresp_author, './/given-names')
    #             corresp_name = surname + given_names
                email = get_xpath_0(corresp_author,'.//email')

                # pmc_id, surname, given_names, email
                cu_mysql.execute('insert into pmc_corresp_author2(pmc_id, surname, given_names, email) \
                     values(%s, %s, %s, %s)',
                     (pmc_id, surname, given_names, email))
        con_mysql.commit()
    except Exception as e:
        con_mysql.rollback()
        cu_mysql.execute('insert into error_pmc2 values(%s, %s)', (term, str(e)))
        con_mysql.commit()


# 多进程处理 pmc数据

import multiprocessing
def multi_process_pmc():
    con_mysql = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
    cu_mysql = con_mysql.cursor()
    cu_pmc_id = con_mysql.cursor()
    cu_pmc_id.execute('select distinct * from pmc_id where pmc_id not in (select distinct pmc_id from pmc_article2)')
    pmc_list = cu_pmc_id.fetchall()    
    pool = multiprocessing.Pool(processes= 10) # 设置进程数
    proc_index = 0
    for pmc_id in pmc_list:
        proc_index += 1
    #     print('complete percent:%2.2f%s'%(proc_index/len(pmc_list)*100,'%'),end='\r') 
        pool.apply_async(process_pmc2, (pmc_id[0], ))

    pool.close()
    pool.join()

# pmc term
def get_pmc_id():
    con_mysql = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
    cu_mysql = con_mysql.cursor()
    cu_mysql.execute('create table if not exists pmc_id(pmc_id varchar(12))')
    cu_mysql.execute('create table if not exists error_pmc_id(page int)')
    con_mysql.commit()
    # con = sqlite3.connect('pubmed.db')
    # cu = con.cursor()
    # # cu.execute('DROP TABLE IF EXISTS dxy')
    # cu.execute('CREATE TABLE IF NOT EXISTS pmid (pmid varchar(12))')
    # cu.execute('CREATE TABLE IF NOT EXISTS error_pmid(page int)')
    # con.commit()

    # 获取pmc id
    term = '(PRC[Affiliation] OR China[Affiliation]) AND ("2015/10/1"[PDat] : "2015/12/31"[PDat]) '
    query_id(term,con_mysql, cu_mysql, db = 'pmc')

    # 获取 pmc 信息

    cu_mysql.close()
    con_mysql.close()

def get_pubmed_id():
    con_mysql = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
    cu_mysql = con_mysql.cursor()
    cu_mysql.execute('create table if not exists pubmed_id(pmc_id varchar(12))')
    cu_mysql.execute('create table if not exists error_pubmed_id(page int)')
    con_mysql.commit()
    # con = sqlite3.connect('pubmed.db')
    # cu = con.cursor()
    # # cu.execute('DROP TABLE IF EXISTS dxy')
    # cu.execute('CREATE TABLE IF NOT EXISTS pmid (pmid varchar(12))')
    # cu.execute('CREATE TABLE IF NOT EXISTS error_pmid(page int)')
    # con.commit()

    # 获取pmc id
    term = '(PRC[Affiliation] OR China[Affiliation]) AND ("2015/10/1"[PDat] : "2015/12/31"[PDat]) '
    query_id(term)
    cu_mysql.close()
    con_mysql.close()

def process_pubmed(term):
    '''
    处理pubmed数据库文章
    :param term:
    :return:
    '''
    try:
        root = efetch(term, db='pubmed')
        articles = root.xpath('//PubmedArticle')
        # 对每一篇文章进行处理
        for article in articles:
            journal = get_xpath_0(article, './/Journal//Title')
    #         journal-id journal-id-type="nlm-journal-id"
            issn = get_xpath_0(article, './/ISSN')
    #         print(issn)
            title = get_xpath_0(article, './/ArticleTitle')
            # subject = article.xpath('.//subject')[0].text
            pmid = get_xpath_0(article, './/PMID')
            doi = get_xpath_0(article, './/ELocationID[@EIdType="doi"]')
            if doi == '0':
                doi = get_xpath_0(article, './/ArticleId[@IdType="doi"]')
            year = get_xpath_0(article, './/PubMedPubDate[@PubStatus="pubmed"]//Year')
            # 文章信息pub-datePubMedPubDate
            # mesh
            if article.xpath('.//MeshHeading//DescriptorName[@MajorTopicYN="N"]'):
                mesh = article.xpath('.//MeshHeading//DescriptorName[@MajorTopicYN="N"]')
                mesh_list = [kw.text for kw in mesh]
                mesh_join = '|'.join(mesh_list)
            else:
                mesh_join='0'
    # keywords
            if article.xpath('.//Keyword[@MajorTopicYN="N"]'):
                keyword = article.xpath('.//Keyword[@MajorTopicYN="N"]')
                keyword_list = [kw.text for kw in keyword]
                keyword_join = '|'.join(keyword_list)
            else:
                keyword_join = '0'
            cu_mysql.execute('insert into  pubmed_articles(pmid, journal, issn, title, doi, year, mesh_join, keyword_join) \
    values(%s, %s, %s, %s,%s, %s, %s, %s)', (pmid, journal, issn, title, doi, year, mesh_join, keyword_join))
            # print(pmid, issn, title, doi, year, mesh_join, keyword_join)
            authors = article.xpath('.//Author[@ValidYN="Y"]')
            author_index = 0
            for author in authors:
                author_index += 1
                surname = get_xpath_0(author, './/LastName')
                given_names = get_xpath_0(author, './/ForeName')
                aff = get_xpath_0(author, './/Affiliation')
                cu_mysql.execute('insert into  pubmed_author(pmid, author_index, surname, given_names, aff) \
    values(%s, %s, %s, %s, %s)',              (pmid, author_index, surname, given_names, aff))
                # print(pmid, author_index, surname, given_names, aff)


    #      以下为第二类

        con_mysql.commit()

    except Exception as e:
        print(e)
        con_mysql.rollback()
        cu_mysql.execute('insert into error_pubmed values(%s, %s)', (term, str(e)))
        con_mysql.commit()

def multi_process_pubmed():
    con_mysql = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
    cu_mysql = con_mysql.cursor()
    # 差集
    cu_mysql.execute('select pmid from pubmed_id where pmid not in (select pmid from pmc_article2) and pmid not in  (select pmid from pubmed_articles);')
    pmid_list = cu_mysql.fetchall()
    pool = multiprocessing.Pool(processes= 10) # 设置进程数
    for pmid in pmid_list:
        pool.apply_async(process_pubmed, (pmid[0], ))
    # process_pubmed('24735618')
    # efetch('24735618')
    pool.close()
    pool.join()
    cu_mysql.close()
    con_mysql.close()

In [ ]:
con_mysql = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu_mysql = con_mysql.cursor()
cu_mysql.execute('create table if not exists pubmed_id_all(pmc_id varchar(12))')
cu_mysql.execute('create table if not exists error_pubmed_id_all(page int)')
con_mysql.commit()
# con = sqlite3.connect('pubmed.db')
# cu = con.cursor()
# # cu.execute('DROP TABLE IF EXISTS dxy')
# cu.execute('CREATE TABLE IF NOT EXISTS pmid (pmid varchar(12))')
# cu.execute('CREATE TABLE IF NOT EXISTS error_pmid(page int)')
# con.commit()

# 获取pmc id
term = '"2011/1/1"[PDat] : "2015/12/31"[PDat] '
query_id(term)
cu_mysql.close()
con_mysql.close()

0/50595
1/50595
2/50595
3/50595
4/50595
5/50595
6/50595
7/50595
8/50595
9/50595
10/50595
11/50595
12/50595
13/50595
14/50595
15/50595